In [6]:
# !pip install kagglehub

import kagglehub
import os
import pandas as pd

# 1) SamSum 다운로드
path = kagglehub.dataset_download("nileshmalode1/samsum-dataset-text-summarization")
print("Path to dataset files:", path)
print(os.listdir(path))

train_raw = pd.read_csv(os.path.join(path, "samsum-train.csv"))
valid_raw = pd.read_csv(os.path.join(path, "samsum-validation.csv"))
test_raw  = pd.read_csv(os.path.join(path, "samsum-test.csv"))

print(train_raw.columns)
print(train_raw.head(5))


Path to dataset files: /data/ephemeral/home/.cache/kagglehub/datasets/nileshmalode1/samsum-dataset-text-summarization/versions/1
['samsum-test.csv', 'samsum-train.csv', 'samsum-validation.csv', 'samsum_dataset']
Index(['id', 'dialogue', 'summary'], dtype='object')
         id                                           dialogue  \
0  13818513  Amanda: I baked  cookies. Do you want some?\r\...   
1  13728867  Olivia: Who are you voting for in this electio...   
2  13681000  Tim: Hi, what's up?\r\nKim: Bad mood tbh, I wa...   
3  13730747  Edward: Rachel, I think I'm in ove with Bella....   
4  13728094  Sam: hey  overheard rick say something\r\nSam:...   

                                             summary  
0  Amanda baked cookies and will bring Jerry some...  
1  Olivia and Olivier are voting for liberals in ...  
2  Kim may try the pomodoro technique recommended...  
3  Edward thinks he is in love with Bella. Rachel...  
4  Sam is confused, because he overheard Rick com...  


In [7]:
import os
import pandas as pd

base = "/data/ephemeral/home/.cache/kagglehub/datasets/nileshmalode1/samsum-dataset-text-summarization/versions/1"
out_dir = "/root/nlp_data/samsum"
os.makedirs(out_dir, exist_ok=True)

train_raw = pd.read_csv(os.path.join(base, "samsum-train.csv"))
valid_raw = pd.read_csv(os.path.join(base, "samsum-validation.csv"))
test_raw  = pd.read_csv(os.path.join(base, "samsum-test.csv"))

# id, dialogue, summary 중에서 dialogue/summary만 쓰고 이름 통일
train_df = train_raw.rename(columns={"dialogue": "dialogue_en", "summary": "summary_en"})[["dialogue_en", "summary_en"]]
valid_df = valid_raw.rename(columns={"dialogue": "dialogue_en", "summary": "summary_en"})[["dialogue_en", "summary_en"]]
test_df  = test_raw.rename(columns={"dialogue": "dialogue_en", "summary": "summary_en"})[["dialogue_en", "summary_en"]]

train_path = os.path.join(out_dir, "samsum_train_en.csv")
valid_path = os.path.join(out_dir, "samsum_valid_en.csv")
test_path  = os.path.join(out_dir, "samsum_test_en.csv")

train_df.to_csv(train_path, index=False)
valid_df.to_csv(valid_path, index=False)
test_df.to_csv(test_path, index=False)

print("✅ 저장 완료:")
print(train_path)
print(valid_path)
print(test_path)


✅ 저장 완료:
/root/nlp_data/samsum/samsum_train_en.csv
/root/nlp_data/samsum/samsum_valid_en.csv
/root/nlp_data/samsum/samsum_test_en.csv


In [8]:
# Cell 3: Solar API 번역 함수 정의

import time
import requests
from tqdm import tqdm
import os
os.environ["SOLAR_API_KEY"] = "up"

SOLAR_API_KEY = os.getenv("SOLAR_API_KEY")
SOLAR_URL = "https://api.upstage.ai/v1/solar/chat/completions"
print("SOLAR_API_KEY:", SOLAR_API_KEY is not None)

# MODEL_NAME = "solar-pro2"

HEADERS = {
    "Authorization": f"Bearer {SOLAR_API_KEY}",
    "Content-Type": "application/json",
}

def call_solar(prompt: str, max_tokens: int = 512, model: str = "solar-1-mini-chat"):
    payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": "You are a professional translator for English and Korean."},
            {"role": "user", "content": prompt},
        ],
        "temperature": 0.1,
        "max_tokens": max_tokens,
    }
    resp = requests.post(SOLAR_URL, headers=HEADERS, json=payload, timeout=60)
    resp.raise_for_status()
    data = resp.json()
    return data["choices"][0]["message"]["content"].strip()

def build_dialogue_prompt(text_en: str) -> str:
    example_ko_dialogue = """#Person1#: 안녕하세요, Mr. Smith. 저는 Dr. Hawkins입니다. 오늘 무슨 일로 오셨어요? 
#Person2#: 건강검진을 받으려고 왔어요. 
#Person1#: 네, 5년 동안 검진을 안 받으셨네요. 매년 한 번씩 받으셔야 해요. 
#Person2#: 알죠. 특별히 아픈 데가 없으면 굳이 갈 필요가 없다고 생각했어요."""

    prompt = f"""
다음은 이 대회에서 사용하는 한국어 대화 형식의 규칙이야.

[대화 스타일 규칙]
- 각 줄은 "#Person1#:", "#Person2#:" 로 시작.
- 화자마다 한 줄씩, 줄바꿈으로 구분.
- 말투는 일상적인 구어체 존댓말 위주.
- 영어 고유명사는 그대로 두고 나머지는 자연스럽게 번역.

[한국어 대화 예시]
{example_ko_dialogue}

위 규칙과 예시를 참고해서, 아래 영어 대화를
동일한 형식의 한국어 대화로 번역해줘.

[영어 대화]
{text_en}

[출력 형식]
- 각 줄을 "#Person1#:" 또는 "#Person2#:" 로 시작.
- 추가 설명 없이 대화문만 출력.
"""
    return prompt


def build_summary_prompt(text_en: str) -> str:
    # 실제 train.csv에서 뽑은 예시
    example_ko_summary = (
        "#Person1#은 #Person2#에게 건강을 위해 담배를 끊을 것을 권유하고, "
        "#Person2#은 이를 어렵다고 느끼지만 동의합니다."
    )

    prompt = f"""
다음은 한국어 대화 요약 데이터셋에서 사용하는 요약문의 스타일 설명이야.

[요약 스타일 규칙]
- 길이: 1~2문장.
- 형식: 화자는 "#Person1#", "#Person2#"와 같이 그대로 사용.
- 내용: 대화에서 가장 중요한 행동과 결정을 요약.
  - 예: 누가 무엇을 요청/설명/제안/동의/거절했는지.
- 톤: 존댓말, 설명체. 감정 묘사는 최소한으로만 사용.
- 대화의 세부 대사를 반복하지 말고, 한 단계 위에서 상황을 정리.

[한국어 요약 예시]
{example_ko_summary}

위 규칙과 예시를 참고해서, 아래 영어 요약 문장을
위와 같은 스타일의 한국어 요약으로 바꿔줘.

[영어 요약]
{text_en}

[출력 형식]
- 1~2문장.
- "#Person1#", "#Person2#" 표기를 그대로 사용.
- 불필요한 설명이나 메타 코멘트 없이 요약문만 출력.
"""
    return prompt



SOLAR_API_KEY: True


In [12]:
def translate_file(in_path: str, out_path: str, batch_size: int = 4, sleep_sec: float = 0.5):
    print(f"📂 입력: {in_path}")
    df = pd.read_csv(in_path)
    
    dialogues_ko = []
    summaries_ko = []
    
    for i in tqdm(range(0, len(df), batch_size), desc=f"Translating {os.path.basename(in_path)}"):
        batch = df.iloc[i:i+batch_size]
        for _, row in batch.iterrows():
            d_en = str(row["dialogue_en"])
            s_en = str(row["summary_en"])
            
            d_ko = call_solar(build_dialogue_prompt(d_en), max_tokens=512)
            s_ko = call_solar(build_summary_prompt(s_en), max_tokens=128)
            
            dialogues_ko.append(d_ko)
            summaries_ko.append(s_ko)
            
            time.sleep(sleep_sec)
    
    df["dialogue_ko"] = dialogues_ko
    df["summary_ko"] = summaries_ko
    df.to_csv(out_path, index=False)
    print(f"✅ 번역 완료 → {out_path}")
    display(df.head(2))


In [9]:
import os
import pandas as pd

SAMSUM_DIR = "/root/nlp_data/samsum"  # 여기로 고정

samsum_train_en = os.path.join(SAMSUM_DIR, "samsum_train_en.csv")
df_sample = pd.read_csv(samsum_train_en).head(1)

text_dialogue_en = df_sample["dialogue_en"].iloc[0]
text_summary_en = df_sample["summary_en"].iloc[0]

print("=== 영어 대화 ===")
print(text_dialogue_en[:300], "\n")

ko_dialogue = call_solar(build_dialogue_prompt(text_dialogue_en), max_tokens=512)
print("=== 번역된 한국어 대화 ===")
print(ko_dialogue, "\n")

ko_summary = call_solar(build_summary_prompt(text_summary_en), max_tokens=128)
print("=== 번역된 한국어 요약 ===")
print(ko_summary)


=== 영어 대화 ===
Amanda: I baked  cookies. Do you want some?
Jerry: Sure!
Amanda: I'll bring you tomorrow :-) 

=== 번역된 한국어 대화 ===
#Person1#: 내가 쿠키 구웠어. 먹을래?
#Person2#: 좋아!
#Person1#: 내일 가져다 줄게 :-) 

=== 번역된 한국어 요약 ===
#Person1#은 오늘 쿠키를 구웠고, 내일 #Person2#에게 그 쿠키를 가져다주기로 했습니다.


In [13]:
# SAMSUM_DIR = "/root/nlp_data/samsum"

# samsum_train_en = os.path.join(SAMSUM_DIR, "samsum_train_en.csv")
# samsum_valid_en = os.path.join(SAMSUM_DIR, "samsum_valid_en.csv")

# 📌 train 일부만 뽑아서 번역
# train_df_sample = pd.read_csv(samsum_train_en).head(500)  # 500개만
# train_df_sample.to_csv(os.path.join(SAMSUM_DIR, "samsum_train_en_500.csv"), index=False)

# samsum_train_en_500 = os.path.join(SAMSUM_DIR, "samsum_train_en_500.csv")
# samsum_train_ko_500 = os.path.join(SAMSUM_DIR, "samsum_train_ko_500.csv")
# samsum_valid_ko = os.path.join(SAMSUM_DIR, "samsum_valid_ko.csv")

# translate_file(samsum_train_en_500, samsum_train_ko_500, batch_size=4, sleep_sec=0.5)
# translate_file(samsum_valid_en, samsum_valid_ko, batch_size=4, sleep_sec=0.5)
SAMSUM_DIR = "/root/nlp_data/samsum"
samsum_valid_en = os.path.join(SAMSUM_DIR, "samsum_valid_en.csv")
samsum_valid_ko = os.path.join(SAMSUM_DIR, "samsum_valid_ko.csv")

translate_file(samsum_valid_en, samsum_valid_ko, batch_size=4, sleep_sec=0.5)


📂 입력: /root/nlp_data/samsum/samsum_valid_en.csv


Translating samsum_valid_en.csv:   0%|          | 0/205 [00:00<?, ?it/s]

Translating samsum_valid_en.csv: 100%|██████████| 205/205 [45:39<00:00, 13.36s/it]

✅ 번역 완료 → /root/nlp_data/samsum/samsum_valid_ko.csv


,dialogue_en,summary_en,dialogue_ko,summary_ko
0,"A: Hi Tom, are you busy tomorrow’s afternoon?\...",A will go to the animal shelter tomorrow to ge...,"#Person1#: 안녕 Tom, 내일 오후에 시간 있어?\n#Person2#: 글...",#Person1#은 아들을 위해 내일 동물 보호소에 가서 강아지를 입양하기로 결정했...
1,Emma: I’ve just fallen in love with this adven...,Emma and Rob love the advent calendar. Lauren ...,#Emma#: 이 어드벤트 캘린더 너무 맘에 들어! 최고야! 우리 애들도 하나 사주...,"#Person1#과 #Person2#는 어드벤트 캘린더를 좋아하며, #Person3..."


In [14]:
# 새 노트북에서 바로 실행 가능
import pandas as pd

train_ko = pd.read_csv("/root/nlp_data/samsum/samsum_train_ko_500.csv")
valid_ko = pd.read_csv("/root/nlp_data/samsum/samsum_valid_ko.csv")

print(f"Train: {len(train_ko)}개")
print(f"Valid: {len(valid_ko)}개")
print("샘플:")
print(train_ko[["dialogue_ko", "summary_ko"]].head(2))


Train: 500개
Valid: 818개
샘플:
                                         dialogue_ko  \
0  #Person1#: 내가 쿠키 구웠어. 먹을래?\n#Person2#: 좋아!\n#P...   
1  #Person1#: 이번 선거에서 누구 뽑을 거야? \n#Person2#: 언제나처...   

                                          summary_ko  
0  #Person1#은 오늘 쿠키를 구웠고, 내일 #Person2#에게 그 쿠키를 가져...  
1  #Person1#과 #Person2#는 이번 선거에서 자유주의자들에게 투표하기로 결...  
